# <font color = 'red'>1. Modeling and Simulation (Experimental Thermal Analysis)

In this module, we will explore the process of thermal analysis modeling, focusing on a real-world example to demonstrate the steps required to achieve experimental data and analyze it for future modeling. Thermal analysis is essential in understanding the behavior of materials under different temperature conditions and is widely used in various fields, including manufacturing, engineering, and material science.

To bring this concept to life, we will present an experimental case where a **thermal video** was captured during the **fused filament fabrication (FFF)** process of **3D printing using PLA (Polylactic Acid)** material. The thermal video provides a detailed visual of the temperature distribution throughout the printing process, showing temperature changes on the surface of the printed material in real-time.

The thermal video contains a temperature range from **30°C to 160°C**, with corresponding color legends indicating different temperature levels. This range offers valuable insights into how heat is dissipated or retained during the printing process, helping to understand how thermal effects influence material properties, layer bonding, and potential warping or deformation of the printed part.

In this module, you will learn:
- How to capture thermal data using infrared or thermal cameras.
- The significance of temperature distribution in 3D printing and its impact on material properties.
- How to analyze this thermal data to create accurate models for thermal behavior prediction.
- How to use these analyses for improving 3D printing processes, including optimizing print settings to reduce defects caused by temperature fluctuations.

## Start a Thermal Video:

In [ ]:
# Import necessary modules
from PIL import Image, ImageDraw, ImageFont
import ipywidgets as widgets
from IPython.display import display
import cv2
import ipywidgets as widgets
from IPython.display import display, Video
import os
from IPython.display import clear_output

# List all video files in the current directory
supported_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.wmv']
files = [f for f in os.listdir('Module 4 Content/Data') if any(f.endswith(ext) for ext in supported_extensions)]

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=files,
    description='Video Files:',
    disabled=False,
)

# Create a button widget
button = widgets.Button(
    description='Select',
    disabled=False,
    button_style='',
    tooltip='Click to select video file',
    icon='check'
)

# Output widget to display messages
output = widgets.Output()

# Function to handle button click
def on_button_click(b):
    global selected_file  # Declare selected_file as global
    with output:
        clear_output()
        selected_file = dropdown.value
        print(f"Video file '{selected_file}' selected.")

# Attach the function to the button widget
button.on_click(on_button_click)

# Display the dropdown, button widgets, and initial message within the output widget
with output:
    print("Please select a video file from the dropdown and click 'Select'.")
display(output)
display(dropdown)
display(button)


## Display Video

In [ ]:
vid_cap = cv2.VideoCapture("Module 4 Content/Data/" + selected_file)

display(Video("Module 4 Content/Data/" + selected_file, embed=True,  width=640, height=480))

### Specify Temperature Range

The thermal camera used for capturing the thermal video has a temperature range spanning from **30°C to 160°C**. This range allows for detailed observation of temperature variations on the printed surface during the **fused filament fabrication (FFF)** process, providing insights into how the PLA material responds to different thermal conditions throughout the 3D printing process.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define text input widgets
text_lowest = widgets.Text(description='Lowest Temperature:',
                           layout=widgets.Layout(width='300px'),
                           style={'description_width': '200px'})
text_highest = widgets.Text(description='Highest Temperature:',
                            layout=widgets.Layout(width='300px'),
                            style={'description_width': '200px'})

# Define a button widget
button = widgets.Button(description='Set Variables')

# Output widget to display messages
output = widgets.Output()

# Function to update global variables lowest and highest
def set_variables(button):
    global lowest, highest
    with output:
        output.clear_output()
        try:
            lowest = int(text_lowest.value)
            highest = int(text_highest.value)
            print(f'Variables set: lowest = {lowest}, highest = {highest}')
        except ValueError:
            print('Error: Please enter valid integer values.')

# Link button click event to function
button.on_click(set_variables)

# Display widgets
display(text_lowest, text_highest, button, output)

## Trim and Crop

When modeling or simulating thermal behavior, we are primarily interested in the area of **thermal exposure**, which represents the regions of the printed part that undergo significant temperature changes during the printing process. Including unnecessary areas outside of this thermal zone can lead to unnecessary computational load, which might slow down processing times without adding valuable data.

To optimize the analysis, it is essential to **crop the image** such that it focuses solely on the region of interest — the area that is being thermally exposed during the entire printing process. This cropped image should cover the **entire part being printed**, from the first frame to the last, ensuring that only relevant data is processed for thermal modeling. By doing this, you eliminate unnecessary background or non-relevant areas, reducing the complexity and improving the efficiency of your simulations.

Set the minute variable to zero and adjust the start time to range from 10 to 40 seconds, ensuring it ends at the desired point.

In [ ]:
from IPython.display import HTML
from IPython.display import IFrame

import os

def display_video(path):
    if os.path.exists(path):
        display(Video(trimmed_video_path, embed=True, width=640, height=480))
    else:
        print(f"Video file not found at {path}")

# Global variable to store the trimmed video path
trimmed_video_path = None

def trim_video_from_cap(vid_cap, start_time, end_time):
    global trimmed_video_path
    if not vid_cap.isOpened():
        with output_widget:
            print("Error: Could not open video.")
        return

    # Get video properties
    fps = int(vid_cap.get(cv2.CAP_PROP_FPS))
    width = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = int(vid_cap.get(cv2.CAP_PROP_FOURCC))
    total_frames = int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate frame numbers for start and end times
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)

    if start_frame < 0 or end_frame > total_frames or start_frame >= end_frame:
        with output_widget:
            print("Error: Invalid start or end time.")
        return

    vid_cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Define the output file path in the current directory
    current_dir = os.getcwd()
    output_folder = os.path.join(current_dir, 'Module 4 Content/Outputs')  # Define the 'Output' folder path
    trimmed_video_path = os.path.join(output_folder, 'trimmed_video_small_slow.mp4')  # Save video in 'Output' folder

    # Define the codec and create VideoWriter object
    out = cv2.VideoWriter(trimmed_video_path, fourcc, fps, (width, height))

    # Read and write frames
    current_frame = start_frame
    while current_frame < end_frame:
        ret, frame = vid_cap.read()
        if not ret:
            break
        out.write(frame)
        current_frame += 1

    # Release everything
    out.release()
    #with output_widget:
        #print(f"Video saved to: {trimmed_video_path}")

# Create widgets for user interaction
start_minute = widgets.IntText(value=0, description='Start Min:', layout=widgets.Layout(width='150px'))
start_second = widgets.IntText(value=0, description='Start Sec:', layout=widgets.Layout(width='150px'))
end_minute = widgets.IntText(value=0, description='End Min:', layout=widgets.Layout(width='150px'))
end_second = widgets.IntText(value=10, description='End Sec:', layout=widgets.Layout(width='150px'))

trim_button = widgets.Button(
    description='Trim Video',
    button_style='success',
    tooltip='Trim the video',
    icon='cut'
)

output_widget = widgets.Output()

# Create layout
start_layout = widgets.HBox([start_minute, start_second])
end_layout = widgets.HBox([end_minute, end_second])
controls_layout = widgets.VBox([start_layout, end_layout, trim_button])

# Display the widgets
display(controls_layout, output_widget)

# Define the button click handler
def handle_trim_button_click(b):
    with output_widget:
        output_widget.clear_output()
        print("Trimming video, please wait...")
    vid_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    start_time = start_minute.value * 60 + start_second.value
    end_time = end_minute.value * 60 + end_second.value
    
    if start_time >= end_time:
        with output_widget:
            print("Error: Start time must be less than end time.")
        return

    trim_video_from_cap(vid_cap, start_time, end_time)

    if trimmed_video_path:
        with output_widget:
            print(f"Video saved to: {trimmed_video_path}")
            print("Displaying video, please wait...")
            display_video(trimmed_video_path)


# Bind the function to the button click event
trim_button.on_click(handle_trim_button_click)


Crop the frames so that the entire part is visible while removing any borders, then slide over the frames to ensure the crop consistently covers the full part throughout the entire sequence.


In [ ]:
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import numpy as np
print("Loading, please wait...")
# Load video using cv7
cap = cv2.VideoCapture(trimmed_video_path)

# Read all frames from the video
frames = []
ret, frame = cap.read()
while ret:
    frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    ret, frame = cap.read()

cap.release()

# Convert frames to PIL images
stored_images = [Image.fromarray(frame) for frame in frames]

# Define sliders and button with bigger text
slider_style = {
    'description_width': 'initial',
    'font_size': '16px'
}
slider_layout = widgets.Layout(width='400px', height='20px')

crop_left = widgets.IntSlider(
    value=0,
    min=0,
    max=700,
    description='Crop Left:',
    continuous_update=True,
    style=slider_style,
    layout=slider_layout
)

crop_right = widgets.IntSlider(
    value=0,
    min=0,
    max=700,
    description='Crop Right:',
    continuous_update=True,
    style=slider_style,
    layout=slider_layout
)

crop_top = widgets.IntSlider(
    value=0,
    min=0,
    max=700,
    description='Crop Top:',
    continuous_update=True,
    style=slider_style,
    layout=slider_layout
)

crop_bottom = widgets.IntSlider(
    value=0,
    min=0,
    max=700,
    description='Crop Bottom:',
    continuous_update=True,
    style=slider_style,
    layout=slider_layout
)

apply_all_button = widgets.Button(
    description='Apply to All',
    tooltip='Apply current settings to all images',
    style={'font_size': '16px'},
    layout=widgets.Layout(width='150px', height='30px')
)

output = widgets.Output()
image_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(stored_images)-1,
    description='Frame',
    continuous_update=True,
    style=slider_style,
    layout=slider_layout
)

# Display function
def display_images(change=None):
    with output:
        
        clear_output(wait=True)
        if stored_images:
            image = stored_images[image_slider.value]
            # Calculate actual crop dimensions
            left = crop_left.value
            top = crop_top.value
            right = crop_right.value
            bottom = crop_bottom.value

            # Ensure crop values are valid
            crop_box = (
                min(left, image.width - 1),
                min(top, image.height - 1),
                max(left + 1, image.width - right),
                max(top + 1, image.height - bottom)
            )
            
            cropped_image = image.crop(crop_box)
            display(cropped_image)

# Apply cropping to all frames
def apply_cropping_to_all(b):
    for i in range(len(stored_images)):
        image = stored_images[i]
        left = crop_left.value
        top = crop_top.value
        right = crop_right.value
        bottom = crop_bottom.value

        # Ensure crop values are valid
        crop_box = (
            min(left, image.width - 1),
            min(top, image.height - 1),
            max(left + 1, image.width - right),
            max(top + 1, image.height - bottom)
        )

        stored_images[i] = image.crop(crop_box)
    with output_widget:
        print("Video Cropped.")

    #display_images()
apply_all_button.on_click(apply_cropping_to_all)

# Observe changes in sliders
crop_left.observe(display_images, 'value')
crop_right.observe(display_images, 'value')
crop_top.observe(display_images, 'value')
crop_bottom.observe(display_images, 'value')
image_slider.observe(display_images, 'value')

# Display initial setup
ui = widgets.VBox([ image_slider, crop_left, crop_right, crop_top, crop_bottom, apply_all_button, output])
display(ui)

# Initial display
display_images()


## Filter out colors

To remove any background noise associated with room temperature in the data, filter out temperatures to ensure they fall between **10°C and 160°C**.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatRangeSlider, Button, VBox, HBox
from PIL import Image
import matplotlib.cm as cm
from IPython.display import display
import ipywidgets as widgets

output_widget = widgets.Output()

# Function to convert PIL images to numpy arrays
def pil_to_array(image):
    return np.array(image)

# Function to display filtered temperature data for a single frame
def display_filtered_image(frame_idx, temp_range):
    image = stored_images[frame_idx]
    filtered_data = filter_image(image, temp_range)
    plt.figure(figsize=(10, 6))
    plt.imshow(filtered_data, cmap='inferno', vmin=lowest, vmax=highest)
    plt.colorbar(label='Temperature (C)')
    plt.title(f'Frame {frame_idx + 1} - Temperature Range {temp_range}C')
    plt.axis('off')
    plt.show()

# Function to filter an image based on temperature range
def filter_image(image, temp_range):
    data = pil_to_array(image)
    # Convert RGB image to grayscale (assuming temperature data is encoded in color)
    if data.ndim == 3:  # Check if the image has color channels
        data = data[:, :, 0] * 0.299 + data[:, :, 1] * 0.587 + data[:, :, 2] * 0.114

    # Normalize data back to the original temperature range
    min_temp, max_temp = 20, 160
    normalized_data = min_temp + (max_temp - min_temp) * (data - data.min()) / (data.max() - data.min())

    # Create a masked array where values outside the range are set to NaN
    filtered_data = np.where((normalized_data >= temp_range[0]) & (normalized_data <= temp_range[1]), normalized_data, np.nan)
    return filtered_data

# Function to apply filtering on all frames and store the results
def apply_all_frames(_):
    global filtered_images
    temp_range = temp_slider.value
    output_widget.clear_output()
    with output_widget:
        print("Processing all frames, please wait...")
    
    filtered_images = [filter_image(image, temp_range) for image in stored_images]

    output_widget.clear_output()
    with output_widget:
        print("Filtering applied to all frames.")

# Create interactive widgets
frame_slider = IntSlider(value=0, min=0, max=len(stored_images) - 1, step=1, description='Frame', layout={'width': '500px'}, style={'description_width': '100px'})
temp_slider = FloatRangeSlider(value=[0, 160], min=0, max=160, step=1, description='Temp Range', layout={'width': '500px'}, style={'description_width': '100px'})
apply_all_button = Button(description='Apply All', layout={'width': '100px'}, style={'description_width': '100px'})

# Link the "Apply All" button with the apply_all_frames function
apply_all_button.on_click(apply_all_frames)

# Create a layout for widgets
ui = VBox([apply_all_button, output_widget])

# Link the display function with the widgets
interact(display_filtered_image, frame_idx=frame_slider, temp_range=temp_slider)
display(ui)


In the following steps, we will specify the points at which we will be collecting the thermal data. Move the **X** and **Y** coordinates until the crosshair is positioned on the bead that is about to be deposited. To move the crosshair to the right, **increase the X coordinate**, and to move it downward, **increase the Y coordinate**. This will ensure that the data is collected at the correct location on the printed bead.


In [ ]:
#Initialize temperature data array
data = []
import numpy as np
from PIL import Image, ImageDraw
import io
import matplotlib.cm as cm
from ipywidgets import interact, IntSlider, VBox, HBox, Output, HTML, IntText, Image as ImageWidget
from IPython.display import display

# Function to convert numpy array to PNG image bytes with a marker
def array_to_png_with_marker(data, x, y, cmap=cm.inferno, vmin=20, vmax=160):
    normed_data = (data - vmin) / (vmax - vmin)
    normed_data = np.clip(normed_data, 0, 1)  # Ensure data is within [0, 1] for colormap
    colored_data = cmap(normed_data)  # Apply colormap
    image = (colored_data[:, :, :3] * 255).astype(np.uint8)  # Convert to RGB
    pil_img = Image.fromarray(image)
    
    # Draw grid around the image
    draw = ImageDraw.Draw(pil_img)
    grid_color = (100, 100, 100)  # Gray color for the grid lines
    for i in range(0, pil_img.width, 100):
        draw.line([(i, 0), (i, pil_img.height)], fill=grid_color, width=1)
    for j in range(0, pil_img.height, 100):
        draw.line([(0, j), (pil_img.width, j)], fill=grid_color, width=1)
    
    # Draw cross marker
    marker_color = (255, 255, 255)  # Bright white color for the marker
    marker_size = 5  # Size of the cross marker
    draw.line([(x - marker_size, y), (x + marker_size, y)], fill=marker_color, width=1)
    draw.line([(x, y - marker_size), (x, y + marker_size)], fill=marker_color, width=1)
    
    with io.BytesIO() as output:
        pil_img.save(output, format='PNG')
        return output.getvalue()


# Create widgets for displaying temperature information
temperature_display = HTML(value="Select coordinates to see the temperature of the pixel.")
x_coord = IntText(value=0, description='X:', layout={'width': '200px'}, style={'description_width': '100px'})
y_coord = IntText(value=0, description='Y:', layout={'width': '200px'}, style={'description_width': '100px'})
output_widget = Output()

# Image widget to display the image
image_widget = ImageWidget(layout={'width': '50%'})

# Function to display the image in the ImageWidget with a marker
def update_image_widget(frame_idx):
    if not filtered_images:
        image_widget.value = b""
        temperature_display.value = "No filtered images to display."
        return
    
    filtered_data = filtered_images[frame_idx]
    x = x_coord.value
    y = y_coord.value
    
    # Ensure coordinates are within bounds before adding marker
    if 0 <= x < filtered_data.shape[1] and 0 <= y < filtered_data.shape[0]:
        png_data = array_to_png_with_marker(filtered_data, x, y)
    else:
        png_data = array_to_png(filtered_data)
    
    image_widget.value = png_data

# Function to update the temperature display based on selected coordinates
def update_temperature_display(*args):
    if not filtered_images:
        temperature_display.value = "No filtered images to display."
        return
    
    frame_idx = frame_slider.value
    filtered_data = filtered_images[frame_idx]
    
    x = x_coord.value
    y = y_coord.value
    
    # Ensure coordinates are within bounds
    if 0 <= x < filtered_data.shape[1] and 0 <= y < filtered_data.shape[0]:
        temp = filtered_data[y, x]
        temperature_display.value = f"Selected Pixel: ({x}, {y})<br>Temperature: {temp:.2f}°C"
        # Update the image with the marker
        update_image_widget(frame_idx)
    else:
        temperature_display.value = "Coordinates are out of bounds."

# Create an interactive slider to select frames
frame_slider = IntSlider(value=0, min=0, max=len(filtered_images) - 1, step=1, description='Frame', layout={'width': '500px'}, style={'description_width': '100px'})

# Link the coordinate selectors to update the temperature display
x_coord.observe(update_temperature_display, names='value')
y_coord.observe(update_temperature_display, names='value')
frame_slider.observe(lambda change: update_image_widget(change['new']), names='value')

# Initial image display
update_image_widget(frame_slider.value)

# Create a layout for the widgets
ui = VBox([frame_slider, HBox([x_coord, y_coord]), temperature_display, image_widget])

# Display the UI
display(ui)


### Display Temperature Values

Make sure to run this step after selecting a set of coordinates where the data will be collected. Once the crosshair is positioned on the desired point, the temperature values at that location will be displayed, allowing you to analyze the thermal data at the specified coordinate.


In [ ]:
import pandas as pd
# Function to extract temperatures for a specific pixel (x, y) across all frames
def extract_pixel_temperatures(x, y, filtered_data):
    for frame_idx, frame_data in enumerate(filtered_data):
        if 0 <= x < frame_data.shape[1] and 0 <= y < frame_data.shape[0]:
            temperature = frame_data[y, x]
            data.append((x, y, frame_idx, temperature))
    return data
x = x_coord.value
y = y_coord.value
pixel_temperatures = extract_pixel_temperatures(x, y, filtered_images)

# Display the results in a DataFrame
df = pd.DataFrame(pixel_temperatures, columns=['X', 'Y', 'Frame Number', 'Temperature'])

# Display the DataFrame
df


### Plot Temperature Values

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the function to plot based on user input
def plot_temperature(x_value, y_value):
    plt.figure(figsize=(8, 6))  # Adjust figure size as needed
    plt.plot(df.loc[(df['X'] == x_value) & (df['Y'] == y_value), "Temperature"])
    plt.xlabel('Index')  # Adjust as needed
    plt.ylabel('Temperature')  # Adjust as needed
    plt.title(f'Temperature Plot for X={x_value}, Y={y_value}')
    plt.grid(True)
    plt.savefig('temp.png')
    plt.show()
    #plt.savefig('temp.png')

# Create ipywidgets for user input
x_widget = widgets.Text(description='X:')
y_widget = widgets.Text(description='Y:')
button = widgets.Button(description="Plot")
output_widget = widgets.Output()

# Define a function to update the output widget with the plot
def on_button_clicked(b):
    with output_widget:
        clear_output()  # Clear previous output
        try:
            x_value = int(x_widget.value)  # Assuming X is numeric
            y_value = int(y_widget.value)  # Assuming Y is numeric
            plot_temperature(x_value, y_value)
        except ValueError:
            print("Please enter valid numeric values for X and Y.")

# Attach the button click event to the function
button.on_click(on_button_clicked)

# Display the widgets and output
display(widgets.HBox([x_widget, y_widget, button]))
display(output_widget)


In [4]:
from IPython.display import display, Video
display(Video("Module 4 Content/Thermal Video.mp4" , embed=True,  width=640, height=480))

### Thermal Curve Analysis: Convection, Conduction, and Radiation

The thermal curve you are observing in the data is the result of a complex interplay between three primary modes of heat transfer: **convection**, **conduction**, and **radiation**. Each of these mechanisms contributes differently to the observed temperature changes over time, and understanding their contributions can offer valuable insights into the behavior of the material and the printing process. Below, we break down the roles of each heat transfer mode in the thermal curve and discuss their influence on the temperature profile, with accompanying model equations.

#### 1. **Convection**

Convection is the transfer of heat between a solid surface and a fluid (in this case, the surrounding air), caused by the movement of the fluid. When you observe the thermal curve decreasing in temperature, it is primarily due to **convection**. As the printed part begins to cool down, the heat is transferred to the surrounding air through the boundary layer adjacent to the surface of the part. 

The rate of heat transfer due to convection can be described using **Newton’s Law of Cooling**:

$$
Q = hA(T_s - T_\infty)
$$

Where:
- $ Q$ is the rate of heat transfer (W)
- $ h$ is the convective heat transfer coefficient (W/m²·K)
- $ A$ is the surface area (m²) of the object (in this case, the printed part)
- $ T_s$ is the surface temperature of the part (K)
- $ T_\infty$ is the ambient temperature of the surrounding air (K)

In this scenario, as the printed part cools down, the temperature difference between the surface and the room temperature decreases, reducing the heat transfer rate. This is why the thermal curve you observe shows a decrease in temperature as convection gradually takes place. The **ambient room temperature** plays a significant role in this process. The thermal energy dissipates through convection as the temperature of the part approaches the ambient temperature of the room.

#### 2. **Conduction**

Conduction is the transfer of heat through a material, occurring from the hotter region to the cooler region within the object. In the case of 3D printing, **conduction** occurs primarily within the layers of the material as the hot filament deposits onto the cooler layers below. This process leads to a **sudden increase in temperature**, which is captured in the thermal curve as a sharp rise in temperature.

The heat transfer due to conduction can be modeled by **Fourier’s Law of Heat Conduction**:

$$
Q = -kA \frac{dT}{dx}
$$

Where:
- $ Q$ is the rate of heat transfer (W)
- $ k$ is the thermal conductivity of the material (W/m·K)
- $ A$ is the cross-sectional area through which heat is conducted (m²)
- $ \frac{dT}{dx}$ is the temperature gradient (K/m) within the material

During the deposition of the upper layer in the 3D printing process, the heat from the molten filament is transferred to the cooler underlying layers. The rate of temperature change in these layers depends on the **thermal conductivity** of the material. PLA, for example, has a certain conductivity that dictates how quickly heat spreads through the material. When the temperature of the upper layer reaches a certain threshold, the heat is rapidly conducted to the layers beneath, resulting in a sharp rise in the thermal curve.

#### 3. **Radiation**

Radiation is the transfer of heat through electromagnetic waves, typically in the infrared spectrum, and does not require a medium (i.e., it can occur in a vacuum). In the 3D printing process, radiation plays a smaller but still important role in heat transfer. The molten filament and the printed part radiate thermal energy to the surrounding environment. The amount of heat radiated can be described by the **Stefan-Boltzmann Law**:

$$
Q = \sigma \epsilon A (T^4 - T_0^4)
$$

Where:
- $ Q$ is the heat radiated per unit time (W)
- $ \sigma$ is the Stefan-Boltzmann constant ($ 5.67 \times 10^{-8} \, \text{W/m}^2 \cdot \text{K}^4$)
- $ \epsilon$ is the emissivity of the surface (a measure of how efficiently the surface radiates energy)
- $ A$ is the surface area (m²)
- $ T$ is the absolute temperature of the object (K)
- $ T_0$ is the absolute temperature of the surrounding environment (K)

As the printed part heats up, it begins radiating energy into the surrounding environment, which causes some of the thermal energy to be lost. While radiation is less significant compared to conduction and convection in this scenario, it still contributes to the overall cooling process, especially as the part cools down and the temperature difference between the part and the environment increases.

#### Conclusion

The observed thermal curve is the combined result of all three modes of heat transfer: convection, conduction, and radiation. The **decrease in temperature** is primarily driven by convection, as the part loses heat to the surrounding air and approaches room temperature. The **sudden increase in temperature** is mainly due to conduction, as heat is transferred to the layers beneath the deposited filament, caused by the conductivity of the material. While radiation also plays a role in dissipating heat, its effect is comparatively smaller but still contributes to the cooling of the part.

By understanding these heat transfer mechanisms, we can better model the thermal behavior of the printed part, optimize printing parameters, and prevent issues such as warping, cracking, or poor layer adhesion. The mathematical models provided by convection, conduction, and radiation equations help us quantify and predict these thermal effects, offering insights into the performance and quality of the 3D printed parts.


Thermal Modeling

In [7]:
from IPython.display import display, Video
display(Video("Module 4 Content/Thermal Simulation.mp4" , embed=True,  width=640, height=480))

<center>
  <img src="Module 4 Content/thermalfit.png" alt="Quantitative vs Qualitative Data" width="500"/>
</center>